In [3]:
import requests as req
import json 
import bs4
import pandas as pd
import os

import concurrent
import datetime

"FileExistsError:"

'FileExistsError:'

In [4]:
test_url="https://finance.yahoo.com/quote/VZ/"
cdx_url_template = ('http://web.archive.org/cdx/search/cdx?url={url}'
                    '&output=json&fl=timestamp,original,statuscode,digest') 
def get_time_df(url):
    with req.get(cdx_url_template.format(url=url)) as r:
        j=r.content
        j=json.loads(j)
    return pd.DataFrame(j[1:],columns=j[0])
get_time_df(test_url)

,timestamp,original,statuscode,digest
0,20160516160413,http://finance.yahoo.com:80/quote/VZ,301,KAV73IUWIQTFXK7YQBGXBLIOEZ6QGEAN
1,20160516160413,http://finance.yahoo.com:80/quote/VZ/,404,GJ32S3AQF57PJWQTMANB7P5FOJ2VSC4S
2,20160517182416,http://finance.yahoo.com:80/quote/VZ,301,L5ZQ2GB52XBRPQNDA7DXTKV7LBS22CJI
3,20160517182416,http://finance.yahoo.com:80/quote/VZ/,404,GJ32S3AQF57PJWQTMANB7P5FOJ2VSC4S
4,20160517190450,http://finance.yahoo.com:80/quote/vz/,404,GG7JELIWXCGDX2M7ABDOZX5SKZLEBPCJ
...,...,...,...,...
845,20221010165659,https://finance.yahoo.com/quote/VZ/,200,ZV4FFKK24ZJUYI7B7LURWXQ2A4F3OHAW
846,20221010165701,https://finance.yahoo.com/quote/VZ,200,MN4GJSQVHVWYNQ5VQ7ZL4YPMJIRMRQVZ
847,20221017211044,https://finance.yahoo.com/quote/VZ,200,QVX43XJQH2BEGRFVAYXLOPSG2KNH2XU2
848,20221018064802,http://finance.yahoo.com/quote/VZ/,200,OHES44XCBETVTDK7WEDZD4BVFSOC435P


In [5]:
wayback="https://web.archive.org/web/"
def get_wayback_url(row):
    return wayback+row["timestamp"]+"/"+row["original"]
def valid_code(code):
    if(code!="-" and 200<=int(code)<300):
        return True
    return False

def get_articles(b):
    for x in b.find_all("li",class_="js-stream-content"):
        for x2 in x.find_all(href=True):
            href=x2["href"]
            if "https://web.archive.org/" not in href:
                href="https://web.archive.org/"+href
            yield (x2.text,x2.parent.parent.text,href)
            
def handle_link(url):
    if "finance.yahoo.com/news/" not in url:
        return None
    try:
        with req.get(url) as r:
            return (bs4.BeautifulSoup(r.content).find("article").text,r.url)
    except Exception:
        print(Exception.__class__.__name__)
        return None
    
def thread_distribute(l,f,n=1):
    #print(type(l))
    if(n==0):
        return f(l)
    
    with concurrent.futures.ThreadPoolExecutor() as executor: 
        l=[executor.submit(lambda x: thread_distribute(x,f,n-1),x) for x in l]
        l= [x.result() for x in l]
    return l

thread_distribute([range(5) for i in range(2)],lambda x: range(x),2)

[[range(0, 0), range(0, 1), range(0, 2), range(0, 3), range(0, 4)],
 [range(0, 0), range(0, 1), range(0, 2), range(0, 3), range(0, 4)]]

In [10]:
file_names=["headline","summary","link"]
def save_page(row,save_adress):
    url=get_wayback_url(row)
    with req.get(url) as r:
        ans=get_articles(bs4.BeautifulSoup(r.content))
    
    for n,article in enumerate(ans):
        folder=os.path.join(save_adress,row["timestamp"],str(n))
        os.makedirs(folder)
        for i in range(3):
            try:
                with open(os.path.join(folder,file_names[i]),mode="w",encoding="utf-8") as f:
                    f.write(article[i])
            except FileExistsError:
                print(f'{os.path.join(row["timestamp"],str(n))} already exists')
                
def colect_stock(ticker):
    base_url=f"https://finance.yahoo.com/quote/{ticker}/"
    
    data=get_time_df(base_url)
    data=data[data["statuscode"].map(valid_code)]
    data=data[data["timestamp"].map(lambda x: int(x[0:6])>=201910)]
    
    thread_distribute(data.iloc[::-1].iloc,lambda x: save_page(x,os.path.join("archive",ticker)))

for stock in ("T","TMUS"):
    print(f"starting {stock}")
    try:
        colect_stock(stock)
    except:
        print(len(next(os.walk(os.path.join("archive",stock)))[1]))

starting T
170
starting TMUS


In [13]:
len(next(os.walk(os.path.join("archive","TMUS")))[1])

58

In [12]:
colect_stock("TMUS")

In [14]:
os.listdir(os.path.join("archive","T"))

['20191012054122',
 '20191012192853',
 '20191013143730',
 '20191021080923',
 '20191026193606',
 '20191028075013',
 '20191215050003',
 '20191227033320',
 '20191228020427',
 '20191229033846',
 '20191230095509',
 '20191231160943',
 '20200101093759',
 '20200101093817',
 '20200102171135',
 '20200126234825',
 '20200131090519',
 '20200201125832',
 '20200202111138',
 '20200203101209',
 '20200204122108',
 '20200212213711',
 '20200311232818',
 '20200314211604',
 '20200331084841',
 '20200407050925',
 '20200413045456',
 '20200417051639',
 '20200429150816',
 '20200506000131',
 '20200519163727',
 '20200527192628',
 '20200602045104',
 '20200602144037',
 '20200612055552',
 '20200618002901',
 '20200620225932',
 '20200625084554',
 '20200701184924',
 '20200702055713',
 '20200704145709',
 '20200726014115',
 '20200730181520',
 '20200804064508',
 '20200805095647',
 '20200806063043',
 '20200809190301',
 '20200810224707',
 '20200816233555',
 '20200817143306',
 '20200903235731',
 '20200905164739',
 '2020090811